# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sechura,-5.5569,-80.8222,21.46,76,21,6.00,PE,1702352680
1,1,port-aux-francais,-49.3500,70.2167,6.05,96,100,7.63,TF,1702352681
2,2,ambodifotatra,-16.9833,49.8500,26.03,94,75,1.03,MG,1702352682
3,3,whitehorse,60.7161,-135.0538,3.43,60,75,10.80,CA,1702352682
4,4,vorgashor,67.5833,63.9500,-16.87,94,100,6.90,RU,1702352539


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:


# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity', hover_cols=['City'])

# Display the map
map_plot.opts(global_extent=True, width=700, height=400)


:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values

min_temp_celsius = (65 - 32) * 5.0/9.0
max_temp_celsius = (85 - 32) * 5.0/9.0

temp_filtered_df = city_data_df[
    (city_data_df['Max Temp'] <= max_temp_celsius) & 
    (city_data_df['Max Temp'] >= min_temp_celsius)
]

# Drop any rows with null values
temp_filtered_df = temp_filtered_df.dropna()
# Display sample data
temp_filtered_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sechura,-5.5569,-80.8222,21.46,76,21,6.00,PE,1702352680
2,2,ambodifotatra,-16.9833,49.8500,26.03,94,75,1.03,MG,1702352682
14,14,pisco,-13.7000,-76.2167,20.03,83,0,5.14,PE,1702352689
17,17,adamstown,-25.0660,-130.1015,21.77,63,66,1.59,PN,1702352509
18,18,ta`u,-14.2336,-169.5144,26.50,88,98,7.63,AS,1702352691
...,...,...,...,...,...,...,...,...,...,...
541,541,mili,6.0815,171.7350,27.85,76,100,8.95,MH,1702353041
543,543,muroto-misakicho,33.2833,134.1500,19.32,83,28,0.68,JP,1702353043
544,544,al hasaheisa,14.7526,33.2984,21.63,18,12,4.58,SD,1702353044
545,545,axim,4.8699,-2.2405,25.95,84,7,1.38,GH,1702353044


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = temp_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,sechura,PE,-5.5569,-80.8222,76,
2,ambodifotatra,MG,-16.9833,49.8500,94,
14,pisco,PE,-13.7000,-76.2167,83,
17,adamstown,PN,-25.0660,-130.1015,63,
18,ta`u,AS,-14.2336,-169.5144,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [48]:
# Set parameters to search for a hotel
radius = 10000  # in meters
params = {
    "apiKey": geoapify_key,  # Replace with your Geoapify API key
    "types": "accommodation.hotel",
    "limit": 1,  # Limit to one result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
sechura - nearest hotel: No hotel found
ambodifotatra - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
canutama - nearest hotel: No hotel found
tangalle - nearest hotel: No hotel found
kabinda - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
daru - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
sibolga - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: No hotel found
lashio - nearest hotel: No hotel found
piripiri - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
gamb

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:


# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE
hotels_found_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]


map_with_hotels = hotels_found_df.hvplot.points(
    'Lng', 'Lat', geo=True, size='Humidity',
    hover_cols=['City', 'Hotel Name', 'Country']
)

# Display the map
map_with_hotels.opts(global_extent=True, width=700, height=400)



:Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)

In [47]:
hotels_found_df

,City,Country,Lat,Lng,Humidity,Hotel Name
